#The Weekend Effect of Mortality in Patients with Acute Kidney Infection

#Imports and Installations

In [1]:
# Data processing libraries
import pandas as pd
import numpy as np
import itertools
import missingno as msno

# Stats libraries
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats

# Image libraries
import matplotlib.pyplot as plt


# Make pandas dataframes prettier
from IPython.display import display, HTML

# Access data using Google BigQuery.
import os
from google.colab import auth
from google.cloud import bigquery
import seaborn as sns

# install and import tableone
!pip install tableone
from tableone import TableOne

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#MIMIC-III

##Connect to MIMIC-III

In [0]:
auth.authenticate_user()

In [0]:
# add your project ID for billing
project_id='physionet-data'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

##Extract data

Data extraction cannot be done directly using Google Colab. For instance, if you try to run:

```
%%bigquery patients

SELECT *
FROM `physionet-data.mimiciii_clinical.patients`;
```

You get the following error:

```
ERROR:
 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/physionet-data/jobs: Access Denied: Project physionet-data: User does not have bigquery.jobs.create permission in project physionet-data.

(job ID: 899daff0-6c46-4450-bf88-48e32ad37cd0)
```

Therefore, the easiest way is to go directly to BigQuery, run the query and save the output on to Google Drive. The tables can then be accessed directly from Google Drive on Colab.
For instance, to get the same information as the query above. 
1. Open [BigQuery](https://console.cloud.google.com/bigquery?pli=1&project=hack-aotearoa&page=jobs)

1.   Enter the SQL query in the Query Editor and click 'Run' (CTRL + Enter):

```
SELECT *
FROM `physionet-data.mimiciii_clinical.patients`;
```

2.   Click 'Save Results' which is located next to Query Results. Give permission to save in CSV format on Google Drive.
2.   Access the file on Google Drive. It will be saved in a time-stamped folder titled 'bq-results-2020...'
2.   Change the file name to 'patients.csv'

##BigQuery SQL queries

icd_diagnosis
```
SELECT *
FROM `physionet-data.mimiciii_clinical.d_icd_diagnoses`
WHERE ICD9_CODE like '99591';
```

Output: 1 row

|Row|	ROW_ID|	ICD9_CODE|	SHORT_TITLE|	LONG_TITLE|
|---|---|---|---|---|
|1	|11403 |99591 | Sepsis |  Sepsis|

diagnosis_icd
```
SELECT *
FROM `physionet-data.mimiciii_clinical.diagnoses_icd`
WHERE ICD9_CODE like '99591';
```
Output: 1,272 rows

|Row|	ROW_ID|	SUBJECT_ID|	HADM_ID|	SEQ_NUM|	ICD9_CODE|
|---|---|---|---|---|---|



angus_sepsis where explicit_sepsis = 1
```
SELECT *
FROM `physionet-data.mimiciii_derived.angus_sepsis`
WHERE explicit_sepsis = 1;
```
Output: 4,085 rows

|Row|	subject_id|	hadm_id|	infection|	explicit_sepsis|	organ_dysfunction|	mech_vent|	angus|
|---|---|---|---|---|---|---|---|	
|1	|15952| 180799| 1 | 1| 0 | 1 | 1 |

explicit_sepsis where sepsis=1
```
SELECT *
FROM `physionet-data.mimiciii_derived.explicit_sepsis`
WHERE sepsis = 1;
```

Output: 4,085 rows

| Row	| subject_id	| hadm_id	| severe_sepsis	| septic_shock	| sepsis |
|---|---|---|---|---|---|
| 1	|21 |111970| 1 | 1 | 1 |


sepsis_cdc_surveillance where sepsis=1


```
SELECT *
FROM `physionet-data.mimiciii_derived.sepsis_cdc_surveillance`
WHERE sepsis = 1;
```
Output: 4,151 rows

|Row	|icustay_id|	sepsis|	sepsis_simple|	suspected_infection_time|	renal|	hematologic|	hepatic|	coagulation|	respiratory|	cardiovascular|	
|---|---|---|---|---|---|---|---|---|---|---|
|1|	214027|      1|       0|               2142-07-12T00:00:00|         0|       1|           0|       0|           0|           0|

sepsis_nqf_500 where sepsis=1

```
SELECT *
FROM `physionet-data.mimiciii_derived.sepsis_nqf_0500`
WHERE sepsis = 1;
```
Output: 6,329

|Row|	icustay_id|	sepsis|	sepsis_dx|	sirs_positive|	organ_failure|	sirs|	cardiovascular|	renal|	hepatic|	hematologic|	coagulation|	metabolism|
|---|---|---|---|---|---|---|---|---|---|---|---|---|	
|1	|208792|1|1|1|1|2|0|0|0|0|0|1|


elixhauser_quan where RENAL_FAILURE=1

```
SELECT *
FROM `physionet-data.mimiciii_derived.elixhauser_quan`
WHERE RENAL_FAILURE = 1;
```
Output: 8,185 rows

|Row|	hadm_id|	CONGESTIVE_HEART_FAILURE|	CARDIAC_ARRHYTHMIAS|	VALVULAR_DISEASE|	PULMONARY_CIRCULATION|	PERIPHERAL_VASCULAR|	HYPERTENSION|	PARALYSIS|	OTHER_NEUROLOGICAL|	CHRONIC_PULMONARY|	DIABETES_UNCOMPLICATED|	DIABETES_COMPLICATED|	HYPOTHYROIDISM|	RENAL_FAILURE|	LIVER_DISEASE|	PEPTIC_ULCER|	AIDS|	LYMPHOMA|	METASTATIC_CANCER|	SOLID_TUMOR|	RHEUMATOID_ARTHRITIS|	COAGULOPATHY|	OBESITY|	WEIGHT_LOSS|	FLUID_ELECTROLYTE|	BLOOD_LOSS_ANEMIA|	DEFICIENCY_ANEMIAS|	ALCOHOL_ABUSE|	DRUG_ABUSE|	PSYCHOSES|	DEPRESSION|	
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|1|100001|0|0|0|0|0|1|0|0|0|0|1|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|


elixhauser_quan

```
SELECT *
FROM `physionet-data.mimiciii_derived.elixhauser_quan`;
```
Output: 58,976 rows

|Row|	hadm_id|	CONGESTIVE_HEART_FAILURE|	CARDIAC_ARRHYTHMIAS|	VALVULAR_DISEASE|	PULMONARY_CIRCULATION|	PERIPHERAL_VASCULAR|	HYPERTENSION|	PARALYSIS|	OTHER_NEUROLOGICAL|	CHRONIC_PULMONARY|	DIABETES_UNCOMPLICATED|	DIABETES_COMPLICATED|	HYPOTHYROIDISM|	RENAL_FAILURE|	LIVER_DISEASE|	PEPTIC_ULCER|	AIDS|	LYMPHOMA|	METASTATIC_CANCER|	SOLID_TUMOR|	RHEUMATOID_ARTHRITIS|	COAGULOPATHY|	OBESITY|	WEIGHT_LOSS|	FLUID_ELECTROLYTE|	BLOOD_LOSS_ANEMIA|	DEFICIENCY_ANEMIAS|	ALCOHOL_ABUSE|	DRUG_ABUSE|	PSYCHOSES|	DEPRESSION|	
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|1|100001|0|0|0|0|0|1|0|0|0|0|1|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|


abx_micro_poe

```
SELECT *
FROM `physionet-data.mimiciii_derived.abx_micro_poe`;
```
Output: 697,920 rows

|Row|	icustay_id|	antibiotic_name|	antibiotic_time|	last72_charttime|	next24_charttime|	suspected_infection_time|	specimen|	positiveculture|
|---|---|---|---|---|---|---|---|---|	
|1|200003|Vancomycin|2199-08-06T00:00:00|2199-08-04T09:53:00|null|2199-08-04T09:53:00|BILE|1|

suspinfect_poe 

```
SELECT *
FROM `physionet-data.mimiciii_derived.suspinfect_poe`;
```
Output: 61,532 rows

|Row|	icustay_id|	antibiotic_name|	antibiotic_time|	suspected_infection_time|	specimen|	positiveculture|	
|---|---|---|---|---|---|---|
|1|200024|Azithromycin |2127-02-21T00:00:00|null|null|null|

suspicion_of_infection 

```
SELECT *
FROM `physionet-data.mimiciii_derived.suspicion_of_infection`;
```

Output: 1,374,912 rows

|Row|	icustay_id|	antibiotic_name|	antibiotic_time|	last72_charttime|	next24_charttime|	suspected_infection_time|	specimen|	positiveculture|
|---|---|---|---|---|---|---|---|---|
|1	|200001|CefTAZidime|2181-11-18T00:00:00|null|2181-11-18T11:45:00|2181-11-18T11:45:00|BLOOD CULTURE|0|

rrtfirstday 

```
SELECT *
FROM `physionet-data.mimiciii_derived.rrtfirstday`;
```
Output: 61,532 rows

|Row|	subject_id|	hadm_id|	icustay_id|	RRT|
|---|---|---|---|---|	
|1|55973|152234|200001|1|

rrt

```
SELECT *
FROM `physionet-data.mimiciii_derived.rrt`;
```
Output: 61,532 rows

|Row|	subject_id|	hadm_id|	icustay_id|	RRT|	
|---|---|---|---|---|
|1 | 55973|152234|200001|1|

crrtdurations 

```
SELECT *
FROM `physionet-data.mimiciii_derived.crrtdurations`;
```
Output: 5,891 rows

|Row|	icustay_id|	num|	starttime|	endtime|	duration_hours|	
|---|---|---|---|---|---|
|1	|null|1|2133-11-08T22:00:00|2133-11-09T11:00:00|13|

dobutaminedurations 

```
SELECT *
FROM `physionet-data.mimiciii_derived.dobutaminedurations`;
```
Output: 1,792 rows

|Row|	icustay_id|	vasonum|	starttime|	endtime|	duration_hours|	
|---|---|---|---|---|---|
|1	|200030|1|2150-11-13T19:30:00|2150-11-15T08:00:00|37|

dopaminedurations

```
SELECT *
FROM `physionet-data.mimiciii_derived.dopaminedurations`;
```
Output: 6,524 rows

|Row|	icustay_id|	vasonum|	starttime|	endtime|	duration_hours|	
|---|---|---|---|---|---|
|1|200024|1|2127-03-03T16:17:00|2127-03-03T20:35:00|4|

norepinephrinedurations 
```
SELECT *
FROM `physionet-data.mimiciii_derived.norepinephrinedurations`;
```
Output: 23,188 rows

|Row|	icustay_id|	vasonum|	starttime|	endtime|	duration_hours|	
|---|---|---|---|---|---|
|1	|200102|1|2193-06-25T07:08:00|2193-06-27T14:00:00|55|2|

phenylephrinedurations
```
SELECT *
FROM `physionet-data.mimiciii_derived.phenylephrinedurations`;
```
Output: 33,141 rows

|Row|	icustay_id|	vasonum|	starttime|	endtime|	duration_hours|	
|---|---|---|---|---|---|
|1	|200003|3|2199-08-04T09:30:00|2199-08-05T10:00:00|25|

milrinonedurations
```
SELECT *
FROM `physionet-data.mimiciii_derived.milrinonedurations`;
```
Output: 3,600 rows

|Row|	icustay_id|	vasonum|	starttime|	endtime|	duration_hours|	
|---|---|---|---|---|---|
|1	|200210|1|2177-07-27T23:45:00|2177-07-28T14:00:00|15|

vasopressindurations

```
SELECT *
FROM `physionet-data.mimiciii_derived.vasopressindurations`;
```
Output: 4,190 rows

|Row|	icustay_id|	vasonum|	starttime|	endtime|	duration_hours|	
|---|---|---|---|---|---|
|1	|null|1|2133-11-07T04:40:00|2133-11-08T09:19:00|29|

fluid_balance

```
SELECT *
FROM `physionet-data.mimiciii_derived.fluid_balance`;
```
Output: 6,794,786 rows

|Row|	icustay_id|	starttime|	endtime|	rate_all|	rate_in|	rate_out|
|---|---|---|---|---|---|---|	
|1	|null|2097-12-07T11:00:00|2097-12-07T12:00:00|214.0|347.3333|133.3333|

uofirstday 

```
SELECT *
FROM `physionet-data.mimiciii_derived.uofirstday`;
```
Output: 53,359 rows

|Row|	subject_id|	hadm_id|	icustay_id|	UrineOutput|
|---|---|---|---|---|	
|1	|3|145834|211552|497.0|

urineoutput

```
SELECT *
FROM `physionet-data.mimiciii_derived.urineoutput`;
```
Output: 3,361,794 rows

|Row|	icustay_id|	charttime|	value|
|---|---|---|---|	
|1	|236980|2174-12-27T21:00:00|175.0|

labevents where ITEMID=50912

```
SELECT *
FROM `physionet-data.mimiciii_clinical.labevents`
WHERE ITEMID=50912;
```

Output: 797,389 rows

|Row|	ROW_ID|	SUBJECT_ID|	HADM_ID|	ITEMID|	CHARTTIME|	VALUE|	VALUENUM|	VALUEUOM|	FLAG|	
|---|---|---|---|---|---|---|---|---|---|
|1	|245182|384|130196|50912|2161-05-23T06:55:00|6.5|6.5|mg/dL|abnormal|